<a href="https://colab.research.google.com/github/jpashami/Data-Analytics-Portfolio/blob/main/OptiDistCondenser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from influxdb_client import InfluxDBClient
import pandas as pd

# Define your InfluxDB credentials and connection details
INFLUXDB_URL = "https://us-east-1-1.aws.cloud2.influxdata.com"
INFLUXDB_TOKEN = "PE-g4ynVLJr_x5KBcXqgyNp8VkFO_Te9otObHu5fqIKc_QLD6Lx2vtLueq-Q77GPgw6avu6QL1Loa5QfIZUOlw=="
INFLUXDB_ORG = "enrisk"
INFLUXDB_BUCKET = "PACE-CC"


In [99]:
# Create a connection to the InfluxDB client
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG, timeout=30000)




In [27]:
test_query = f'''
from(bucket: "{INFLUXDB_BUCKET}")
  |> range(start: -1h)
  |> limit(n: 5)
'''

try:
    test_tables = client.query_api().query(test_query)
    for table in test_tables:
        for record in table.records:
            print(record)
except Exception as e:
    print(f"Error during test query: {e}")

In [109]:
import time
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient

# Define the start and end dates
start_date = datetime(2023, 9, 1)
end_date = datetime(2024, 9, 1)

# Iterate over smaller time ranges
current_start = start_date
results = []
chunk_counter = 0  # Track number of chunks processed

while current_start < end_date:
    chunk_counter += 1
    current_end = min(current_start + timedelta(days=7), end_date)

    # Format start and end times in ISO format with "Z" for UTC
    start_time_utc = current_start.isoformat() + "Z"
    end_time_utc = current_end.isoformat() + "Z"

    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: {start_time_utc}, stop: {end_time_utc})
      |> filter(fn: (r) => r._measurement == "PACOptiDistJSONInputResult")
      |> filter(fn: (r) => r.siteID == "2")
    '''

    print(f"Processing chunk {chunk_counter}: {start_time_utc} to {end_time_utc}")

    try:
        # Execute the query
        result = client.query_api().query(query=query, org=INFLUXDB_ORG)
        records_count = sum(len(table.records) for table in result)
        results.extend(result)  # Append results

        print(f"Chunk {chunk_counter} completed. Retrieved {records_count} records.")
    except Exception as e:
        print(f"Chunk {chunk_counter} failed with error: {e}")

    # Move to the next time range
    current_start = current_end

# Final summary
total_records = sum(len(table.records) for table in results)
print(f"All chunks processed. Total chunks: {chunk_counter}. Total records retrieved: {total_records}.")

Processing chunk 1: 2023-09-01T00:00:00Z to 2023-09-08T00:00:00Z
Chunk 1 completed. Retrieved 856 records.
Processing chunk 2: 2023-09-08T00:00:00Z to 2023-09-15T00:00:00Z
Chunk 2 completed. Retrieved 766 records.
Processing chunk 3: 2023-09-15T00:00:00Z to 2023-09-22T00:00:00Z
Chunk 3 completed. Retrieved 855 records.
Processing chunk 4: 2023-09-22T00:00:00Z to 2023-09-29T00:00:00Z
Chunk 4 completed. Retrieved 842 records.
Processing chunk 5: 2023-09-29T00:00:00Z to 2023-10-06T00:00:00Z
Chunk 5 completed. Retrieved 838 records.
Processing chunk 6: 2023-10-06T00:00:00Z to 2023-10-13T00:00:00Z
Chunk 6 completed. Retrieved 900 records.
Processing chunk 7: 2023-10-13T00:00:00Z to 2023-10-20T00:00:00Z
Chunk 7 completed. Retrieved 992 records.
Processing chunk 8: 2023-10-20T00:00:00Z to 2023-10-27T00:00:00Z
Chunk 8 completed. Retrieved 810 records.
Processing chunk 9: 2023-10-27T00:00:00Z to 2023-11-03T00:00:00Z
Chunk 9 completed. Retrieved 834 records.
Processing chunk 10: 2023-11-03T00:00

In [104]:
test_query = f'''
from(bucket: "{INFLUXDB_BUCKET}")
  |> range(start: 2024-08-30T00:00:00Z, stop: 2024-09-01T00:00:00Z)
  |> filter(fn: (r) => r._measurement == "PACOptiDistJSONInputResult")
  |> filter(fn: (r) => r.siteID == "2")
'''
print(client.query_api().query(query=test_query, org=INFLUXDB_ORG))


[<FluxTable: 11 columns, 12 records>, <FluxTable: 11 columns, 12 records>, <FluxTable: 11 columns, 11 records>, <FluxTable: 11 columns, 11 records>, <FluxTable: 11 columns, 10 records>, <FluxTable: 11 columns, 10 records>, <FluxTable: 11 columns, 15 records>, <FluxTable: 11 columns, 15 records>]


In [96]:
# Define your query to retrieve the JSON string from the 'data' column
query = f'''
from(bucket: "PACE-CC")
  |> range(start: -90d, stop: now())
  |> filter(fn: (r) => r._measurement == "PACOptiDistJSONInputResult")
  |> filter(fn: (r) => r.siteID == "2")
  |> limit(n: 10)
'''
count = 0
try:
    tables = client.query_api().query(query)
    for table in tables:
        for record in table.records:
            #print(record)
            count = count + 1
            print(record)
except Exception as e:
    print(f"Error during query: {e}")
print(count)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 9, 21, 18, 56, 39, 843211, tzinfo=tzlocal()), '_stop': datetime.datetime(2024, 12, 20, 18, 56, 39, 843211, tzinfo=tzlocal()), '_time': datetime.datetime(2024, 9, 21, 20, 50, 44, 567000, tzinfo=tzlocal()), '_value': '{"OptiDist":{"version":"1","Response":{"command":"Get","argument":"Result"},"Result":{"chMessage":"Nothing to send"}}}', '_field': 'data', '_measurement': 'PACOptiDistJSONInputResult', 'instrument': '08166', 'labID': '1', 'siteID': '2'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 9, 21, 18, 56, 39, 843211, tzinfo=tzlocal()), '_stop': datetime.datetime(2024, 12, 20, 18, 56, 39, 843211, tzinfo=tzlocal()), '_time': datetime.datetime(2024, 9, 21, 23, 58, 44, 37000, tzinfo=tzlocal()), '_value': '{"OptiDist":{"version":"1","Response":{"command":"Get","argument":"Result"},"Result":{"chMessage":"Nothing to send"}}}', '_field': 'data', '_measureme

In [110]:
print(results[0].records[3])


FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 9, 1, 0, 0, tzinfo=tzlocal()), '_stop': datetime.datetime(2023, 9, 8, 0, 0, tzinfo=tzlocal()), '_time': datetime.datetime(2023, 9, 1, 19, 19, 10, 852000, tzinfo=tzlocal()), '_value': '{"OptiDist":{"version":"1","Response":{"command":"Get","argument":"Result"},"Result":{"CommonResultData":{"chInstrumentName":"24585 OPTIDIST #7","chSerialNumber":"08166","lInstrumentID":"1","chSampleNo":"180415-6","chOperator":"PAC user","chNotes":{},"tTimeAtStart":"1693536600","tTimeAtEnd":"1693536600","lEndType":"131072","lAmbientTmpCAtStart":"-2147483648","lAirPressureAtStart":"101320","lCondenserTmpCAtStart":"-2147483648","lReceiverTmpCAtStart":"-2147483648","lChargeVolume":"100000","l5ccResVolume":"-2147483648","lFBPVolume":"-2147483648","lFBPVapTemp":"-2147483648","nFBPTime":"-32768","lDPVolume":"-2147483648","lDPVapTemp":"-2147483648","nDPTime":"-32768","lSTOPVolume":"3919","lSTOPVapTemp":"18467","nSTOPTime":

In [111]:
import json
import pandas as pd

# Parse the JSON payload and extract test-related fields
records = []

for table in results:
    for record in table.records:
        try:
            # Access `_value` directly as a key
            _value = record["_value"]
            if not isinstance(_value, str):
                print(f"Skipping record due to invalid `_value`: {_value}")
                continue

            # Parse the JSON string in the `_value` column
            json_payload = json.loads(_value)

            # Skip records with a specific message
            ch_message = json_payload.get("OptiDist", {}).get("Result", {}).get("chMessage")
            if ch_message == "Nothing to send":
                print(f"Skipping record due to 'Nothing to send' message: {_value}")
                continue

            # Safely navigate through nested JSON structure
            optidist_result = json_payload.get("OptiDist", {}).get("Result", {})
            product_data = optidist_result.get("Product", {})
            common_data = optidist_result.get("CommonResultData", {})
            standard_data = optidist_result.get("Standard", {})

            # Extract test-related fields
            condenser_data = {
                "_time": record["_time"],  # Timestamp of the record
                "instrumentName": common_data.get("chInstrumentName"),  # Instrument name
                "sampleName": common_data.get("chSampleNo"),  # Sample name
                "productName": product_data.get("chProfileName"),  # Product name
                "standard": standard_data.get("chStdName"),  # Standard name
                "testStartTime": common_data.get("tTimeAtStart"),  # Test start time
                "testStopTime": common_data.get("tTimeAtEnd"),  # Test stop time
                "labID": record["labID"],  # Lab ID from the record
                "siteID": record["siteID"],  # Site ID from the record
                "Result_lCondenserTmpCAtStart": common_data.get("lCondenserTmpCAtStart"),
                "Product_lCondenserTempStart": product_data.get("lCondenserTempStart"),
                "Product_lCondenserTempSwitch": product_data.get("lCondenserTempSwitch"),
                "Product_lCondenserTempEnd": product_data.get("lCondenserTempEnd"),
                "Standard_lCondenserTempMinC": standard_data.get("lCondenserTempMinC"),
                "Standard_lCondenserTempMaxC": standard_data.get("lCondenserTempMaxC"),
                "Standard_lCondenserTempMinF": standard_data.get("lCondenserTempMinF"),
                "Standard_lCondenserTempMaxF": standard_data.get("lCondenserTempMaxF"),
                "Product_nCondenserMode": product_data.get("nCondenserMode"),
            }
            records.append(condenser_data)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for record: {_value}, Error: {e}")
        except KeyError as e:
            print(f"KeyError: {e}, record: {record}")
        except Exception as e:
            print(f"Unexpected error processing record: {record}, Error: {e}")

# Create a DataFrame for the parsed data
df_condenser = pd.DataFrame(records)


Streaming output truncated to the last 5000 lines.
Skipping record due to invalid `_value`: 1721694957627.0
Skipping record due to invalid `_value`: 1721696037910.0
Skipping record due to invalid `_value`: 1721696398708.0
Skipping record due to invalid `_value`: 1721696877255.0
Skipping record due to invalid `_value`: 1721699277288.0
Skipping record due to invalid `_value`: 1721699457933.0
Skipping record due to invalid `_value`: 1721702757547.0
Skipping record due to invalid `_value`: 1721702997062.0
Skipping record due to invalid `_value`: 1721707377461.0
Skipping record due to invalid `_value`: 1721708158022.0
Skipping record due to invalid `_value`: 1721710018255.0
Skipping record due to invalid `_value`: 1721715657536.0
Skipping record due to invalid `_value`: 1721716917366.0
Skipping record due to invalid `_value`: 1721717457710.0
Skipping record due to invalid `_value`: 1721718537437.0
Skipping record due to invalid `_value`: 1721725197435.0
Skipping record due to invalid `_valu

In [112]:
df_condenser.head()

,_time,instrumentName,sampleName,productName,standard,testStartTime,testStopTime,labID,siteID,Result_lCondenserTmpCAtStart,Product_lCondenserTempStart,Product_lCondenserTempSwitch,Product_lCondenserTempEnd,Standard_lCondenserTempMinC,Standard_lCondenserTempMaxC,Standard_lCondenserTempMinF,Standard_lCondenserTempMaxF,Product_nCondenserMode
0,2023-09-01 07:48:38.325000+00:00,24585 OPTIDIST #7,180415-6,AUTO G1+G2 GAS EVAP,ASTM D 86 - G1,1693536335,1693536335,1,2,-2147483648,500,400000,32900,0,1000,32000,34000,2
1,2023-09-01 07:50:38.390000+00:00,24585 OPTIDIST #7,180415-6,AUTO G1+G2 GAS EVAP,ASTM D 86 - G1,1693536544,1693536544,1,2,-2147483648,500,400000,32900,0,1000,32000,34000,2
2,2023-09-01 19:17:11.022000+00:00,24585 OPTIDIST #7,180415-6,AUTO G1+G2 GAS EVAP,ASTM D 86 - G1,1693536642,1693536642,1,2,-2147483648,500,400000,32900,0,1000,32000,34000,2
3,2023-09-01 19:19:10.852000+00:00,24585 OPTIDIST #7,180415-6,AUTO G1+G2 GAS EVAP,ASTM D 86 - G1,1693536600,1693536600,1,2,-2147483648,500,400000,32900,0,1000,32000,34000,2
4,2023-09-01 19:24:18.617000+00:00,24585 OPTIDIST #7,180415-6,AUTO G1+G2 GAS EVAP,ASTM D 86 - G1,1693536499,1693536499,1,2,-2147483648,500,400000,32900,0,1000,32000,34000,2


In [113]:
df_condenser.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12905 entries, 0 to 12904
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype                    
---  ------                        --------------  -----                    
 0   _time                         12905 non-null  datetime64[ns, tzlocal()]
 1   instrumentName                12905 non-null  object                   
 2   sampleName                    12905 non-null  object                   
 3   productName                   12905 non-null  object                   
 4   standard                      12905 non-null  object                   
 5   testStartTime                 12905 non-null  object                   
 6   testStopTime                  12905 non-null  object                   
 7   labID                         12905 non-null  object                   
 8   siteID                        12905 non-null  object                   
 9   Result_lCondenserTmpCAtStart  12905 non

In [114]:
# Save to CSV
df_condenser.to_csv("condenser_data_test.csv", index=False)

In [115]:
from google.colab import files

# Download the CSV file
files.download('condenser_data_test.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
df_condenser["Standard_lCondenserTempMaxC"].count()

12905

In [117]:
df_condenser["Standard_lCondenserTempMaxC"].isna().sum()

0

In [119]:
# Count of items in each category (<=7000 and >7000) for "Standard_lCondenserTempMaxC"
# Ensure the `Standard_lCondenserTempMaxC` column is of numeric type
df_condenser["Standard_lCondenserTempMaxC"] = pd.to_numeric(df_condenser["Standard_lCondenserTempMaxC"], errors="coerce")
category_counts = {
    "<=7000": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] <= 7000].shape[0],
    ">7000": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] > 7000].shape[0],
}

category_counts

{'<=7000': 10278, '>7000': 2627}

In [120]:
# Categorize and count unique standards and products based on `Standard_lCondenserTempMaxC`
threshold = 7000
# Ensure the `Standard_lCondenserTempMaxC` column is of numeric type
df_condenser["Standard_lCondenserTempMaxC"] = pd.to_numeric(df_condenser["Standard_lCondenserTempMaxC"], errors="coerce")

# Remove rows with NaN values in the `Standard_lCondenserTempMaxC` column after conversion
#df_condenser = df_condenser.dropna(subset=["Standard_lCondenserTempMaxC"])

# Re-categorize and calculate the summary
df_condenser["Category"] = df_condenser["Standard_lCondenserTempMaxC"].apply(
    lambda x: "<= 7000" if x <= threshold else "> 7000"
)

category_summary = (
    df_condenser.groupby("Category")
    .agg(
        unique_standards=("standard", lambda x: len(x.unique())),
        unique_products=("productName", lambda x: len(x.unique())),
    )
    .reset_index()
)

print("Condenser Temperature Category Summary:")
print(category_summary)

Condenser Temperature Category Summary:
  Category  unique_standards  unique_products
0  <= 7000                 3              475
1   > 7000                 3               43


In [121]:
# Recalculate the summary with corrected column names
summary = {
    "<=7000": {
        "unique_standards": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] <= 7000]["standard"].unique().tolist(),
        #"unique_products": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] <= 7000]["productName"].unique().tolist(),
        "count": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] <= 7000].shape[0]
    },
    ">7000": {
        "unique_standards": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] > 7000]["standard"].unique().tolist(),
        #"unique_products": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] > 7000]["productName"].unique().tolist(),
        "count": df_condenser[df_condenser["Standard_lCondenserTempMaxC"] > 7000].shape[0]
    }
}

summary
# Create a DataFrame
summary_df = pd.DataFrame(summary)

# Display the summary table
print(summary_df)

                                                            <=7000  \
unique_standards  [ASTM D 86 - G1, ASTM D 86 - G2, ASTM D 86 - G3]   
count                                                        10278   

                                                              >7000  
unique_standards  [ASTM D 86 - G4, ASTM D 850 IBP<110C, ASTM D 8...  
count                                                          2627  


In [122]:
display(summary_df)

,<=7000,>7000
unique_standards,"[ASTM D 86 - G1, ASTM D 86 - G2, ASTM D 86 - G3]","[ASTM D 86 - G4, ASTM D 850 IBP<110C, ASTM D 8..."
count,10278,2627
